In [1]:
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAI
from langchain.document_loaders import JSONLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
import os
from dotenv import load_dotenv
load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
#print(os.getenv("GOOGLE_API_KEY"))

d:\my_code_profile\group_project\Rag_Base\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#print(openai_api_key)

folder = r"D:\my_code_profile\group_project\Rag_Base\jsonfiles\food"
documents = []

print(f"Attempting to access folder: '{folder}'") # Add this line
if not os.path.exists(folder): # Add this check
    print(f"Error: Folder '{folder}' does not exist.")
else:
    print(f"Folder '{folder}' exists. Listing contents...")

for file in os.listdir(folder):
    if file.endswith(".json"):
        f = os.path.join(folder, file)
        loader= JSONLoader(file_path=f, jq_schema = ".", text_content=False )  
        documents.extend(loader.load())

print(f"Loaded {len(documents)} pages")

Attempting to access folder: 'D:\my_code_profile\group_project\Rag_Base\jsonfiles\food'
Folder 'D:\my_code_profile\group_project\Rag_Base\jsonfiles\food' exists. Listing contents...
Loaded 1 pages


In [3]:
faiss_index_dir = "faiss_index"  # You can change this path as needed

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
if not os.path.exists(faiss_index_dir):
    os.makedirs(faiss_index_dir)
    print(f"Created FAISS index directory: {faiss_index_dir}")

faiss_main_file = os.path.join(faiss_index_dir, "index.faiss")
faiss_pkl_file = os.path.join(faiss_index_dir, "index.pkl")

if os.path.exists(faiss_main_file) and os.path.exists(faiss_pkl_file):
    # If index exists, load it
    vectorstore = FAISS.load_local(faiss_index_dir, embedding_model, allow_dangerous_deserialization=True)
    print("FAISS Document Index Loaded from existing directory.")
else:
    # If index does not exist, create it and save it
    print("Creating and saving FAISS Document Index...")
    vectorstore = FAISS.from_documents(documents, embedding_model)
    vectorstore.save_local(faiss_index_dir)
    print("FAISS Document Index Created and Saved.")

retriever = vectorstore.as_retriever()


FAISS Document Index Loaded from existing directory.


In [4]:

template = '''
Chat History : {chat_history}
Context : {context}
Question : {question}
Instructions:
1. For the food emisson tracker you have to use the below context:
-> Apply the appropriate formula to calculate the CO₂ amount based on the question
use this formula: Total = quantity x emission factor,  where user will give quantity and material type. from that get average weight from context.
-> Output the CO₂ amount (in kg).
-> Also give 2 suggestions to reduce this emission.

'''
custom_prompt = PromptTemplate.from_template(template)

# --- Memory ---
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


C:\Users\HP\AppData\Local\Temp\ipykernel_10592\2287625772.py:16: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

conversational_qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever = retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_prompt},
    return_source_documents=False 
)

In [6]:
#print(os.getenv("GOOGLE_API_KEY"))
print("\n--- RAG Chatbot (with Conversation History) ---")
print("Type 'exit', 'quit', or 'bye' to end the conversation.")

while True:
    try:
        user_query = input("You: ")
        if user_query.lower() in ["exit", "quit", "bye"]:
            print("Bot: Goodbye!")
            break

 
        response = conversational_qa_chain({"question": user_query})
        print(f"Bot: {response['answer']}")

    except Exception as e:
        print(f"An error occurred: {e}")
        break


--- RAG Chatbot (with Conversation History) ---
Type 'exit', 'quit', or 'bye' to end the conversation.


C:\Users\HP\AppData\Local\Temp\ipykernel_10592\795605312.py:13: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = conversational_qa_chain({"question": user_query})


Bot: To calculate the carbon emission for "1 plate chhole bhature", we need to estimate the quantity of its main ingredients, as "chhole bhature" as a composite dish is not directly listed in the provided context.

Based on typical serving sizes, we can estimate:
*   **Chickpeas (Chhole):** Approximately 0.2 kg
*   **Wheat (for Bhature):** Approximately 0.2 kg (flour equivalent)

Now, let's use the emission factors from the context:
*   Emission factor for chickpeas: 1 kg CO₂e per kg
*   Emission factor for wheat: 2 kg CO₂e per kg

**Calculation:**
*   **Chhole (chickpeas):** 0.2 kg * 1 kg CO₂e/kg = 0.2 kg CO₂e
*   **Bhature (wheat):** 0.2 kg * 2 kg CO₂e/kg = 0.4 kg CO₂e

**Total estimated CO₂ emission for 1 plate chhole bhature = 0.2 kg + 0.4 kg = 0.6 kg CO₂e**

**Suggestions to reduce food emissions:**

1.  **Prioritize plant-based meals:** While chhole bhature is already vegetarian, incorporating more plant-based dishes with ingredients like lentils, beans, and vegetables (which gen

KeyboardInterrupt: 